Installations

In [ ]:
%pip install pandas numpy

Imports

In [ ]:
import subprocess
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2e}'.format

Getting BenchGen Perf

In [ ]:
iterations = 3
examples_folder_path = '../examplesGenerator/'
subprocess.run(f"mkdir {examples_folder_path}results", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"rm {examples_folder_path}results/*", shell=True)
subprocess.run(f"cd {examples_folder_path} && python3 examplesGenerator.py 1 {iterations}", shell=True, check=True)

perf_record_folder_path = '../perfRecord/'
subprocess.run(f"cd {perf_record_folder_path} && python3 runPerf.py {examples_folder_path}results/ clang++", shell=True, check=True)
subprocess.run(f"mkdir {perf_record_folder_path}results", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"cp {perf_record_folder_path}results/results.csv ./data/benchGenPerf.csv  ", shell=True, check=True)

Perf files

In [ ]:
bench_gen_perf_file = "./data/benchGenPerf.csv"
bench_gen_perf = pd.read_csv(bench_gen_perf_file)

llvm_test_suite_perf_file = "./data/llvmTestSuitePerf.csv"
llvm_test_suite_perf = pd.read_csv(llvm_test_suite_perf_file)

spec_2017_perf_file = "./data/spec2017Perf.csv"
spec_2017_perf = pd.read_csv(spec_2017_perf_file)

Perf events

In [ ]:
llvm_spec_intersection = llvm_test_suite_perf.columns.intersection(spec_2017_perf.columns)
perf_events = bench_gen_perf.columns.intersection(llvm_spec_intersection)

Filtering dataframes

In [ ]:
llvm_test_suite_perf_filtered = llvm_test_suite_perf[perf_events]
spec_2017_perf_filtered = spec_2017_perf[perf_events]
bench_gen_perf_filtered = bench_gen_perf[perf_events]
bench_gen_perf_filtered = bench_gen_perf_filtered.apply(pd.to_numeric, errors='coerce')

Centroids

In [ ]:
llvm_test_suite_centroids = llvm_test_suite_perf_filtered.mean().round()
spec_2017_centroids = spec_2017_perf_filtered.mean().round()
bench_gen_centroids = bench_gen_perf_filtered.mean().round()

Distances

In [ ]:
llvm_distances_to_llvm_centroid = llvm_test_suite_perf_filtered - llvm_test_suite_centroids
llvm_average_distances_to_llvm_centroid = llvm_distances_to_llvm_centroid.abs().mean().round()
spec_distances_to_spec_centroid = spec_2017_perf_filtered - spec_2017_centroids
spec_average_distances_to_spec_centroid = spec_distances_to_spec_centroid.abs().mean().round()
bench_gen_distances_to_llvm_centroid = bench_gen_centroids - llvm_test_suite_centroids
bench_gen_distances_to_spec_centroid = bench_gen_centroids - spec_2017_centroids

LLVM test suite results

In [ ]:
print("LLVM centroids:")
print(llvm_test_suite_centroids)
print("===========================================================")
print("LLVM average distances to LLVM centroid:")
print(llvm_average_distances_to_llvm_centroid)

SPEC 2017 results

In [ ]:
print("SPEC 2017 centroids:")
print(spec_2017_centroids)
print("===========================================================")
print("SPEC 2017 average distances to SPEC 2017 centroid:")
print(spec_average_distances_to_spec_centroid)

BenchGen results

In [ ]:
print("BenchGen centroids:")
print(bench_gen_centroids)

BenchGen X LLVM test suite results

In [ ]:
print("BenchGen distance to LLVM centroid:")
print(bench_gen_distances_to_llvm_centroid)

BenchGen X SPEC 2017 results

In [ ]:
print("BenchGen distance to SPEC 2017 centroid:")
print(bench_gen_distances_to_spec_centroid)

Generate CSV

In [ ]:
row0 = pd.Index(perf_events, name="Performance Events")
row1 = pd.Index(llvm_test_suite_centroids, name="LLVM centroids")
row2 = pd.Index(spec_2017_centroids, name="SPEC 2017 centroids")
row3 = pd.Index(bench_gen_centroids, name="BenchGen centroids")
row4 = pd.Index(llvm_average_distances_to_llvm_centroid, name="LLVM average distance to LLVM centroids")
row5 = pd.Index(bench_gen_distances_to_llvm_centroid, name="BenchGen distance to LLVM centroids")
row6 = pd.Index(spec_average_distances_to_spec_centroid, name="SPEC average distance to SPEC 2017 centroids")
row7 = pd.Index(bench_gen_distances_to_spec_centroid, name="BenchGen distance to SPEC 2017 centroids")

s0 = pd.Series(row0, name=row0.name)
s1 = pd.Series(row1, name=row1.name)
s2 = pd.Series(row2, name=row2.name)
s3 = pd.Series(row3, name=row3.name)
s4 = pd.Series(row4, name=row4.name)
s5 = pd.Series(row5, name=row5.name)
s6 = pd.Series(row6, name=row6.name)
s7 = pd.Series(row7, name=row7.name)

df = pd.DataFrame({s0.name : s0, s1.name: s1, s2.name: s2, s3.name: s3, s4.name: s4, s5.name: s5, s6.name: s6, s7.name: s7})

df = df.transpose()

subprocess.run(f"mkdir results", shell=True, stderr=subprocess.DEVNULL)
df.to_csv("./results/analysisResult.csv", float_format='%.0f')